In [1]:
import openai
import json
import numpy as np
from numpy.linalg import norm
import requests
import csv
import tiktoken
from pprint import pprint
api_key = open("api_key").read()
openai.api_key = api_key
def get_embedding(text, model="text-embedding-ada-002"):
   encoder = tiktoken.encoding_for_model(model)
   token_length = len(encoder.encode(text))
   while token_length > 8191:
      text = text[:-100]
      token_length = len(encoder.encode(text))
      print(token_length)
   # text = text.replace("\n", " ")
   # return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
   url = 'https://api.openai.com/v1/embeddings'
   headers = {
      'Content-Type': 'application/json',
      'Authorization': "Bearer {}".format(api_key)
   }
   data = {
      "input": text,
      "model": model
   }
   res = requests.post(url, headers=headers, json=data)
   res = res.json()
   return res['data'][0]['embedding']

def cos_sim(a, b):
   return np.dot(a, b)/(norm(a)*norm(b))

def save_json(data, filepath=r'new_data.json'):
   with open(filepath, 'w') as fp:
      json.dump(data, fp, indent=4)

In [1]:
import json
from collections import defaultdict
pairwise_evaluation = json.load(open('pairwise_evaluation_w_embeddings.json'))
pairwise_evaluation_dict = defaultdict(list)
for item in pairwise_evaluation:
    pairwise_evaluation_dict[item['article_id']].append(item)

In [4]:
from pprint import pprint
writer_samples = {}
for article_id, data in pairwise_evaluation_dict.items():
    writer_samples[article_id] = data[0]
save_json(writer_samples, 'pairwise_samples.json')


In [2]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from scipy import spatial

def estimate_transformation(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    if np.linalg.matrix_rank(X) == X.shape[0]:
        # X is invertible, so solve for A
        A = Y @ np.linalg.inv(X)
        print("Matrix A:", A)
    else:
        X = X.reshape((-1, X.shape[-1]))
        Y = Y.reshape((-1, Y.shape[-1]))
        A, residuals, _, _ = np.linalg.lstsq(X, Y, rcond=None)
        result = np.allclose(np.dot(X, A).T, Y.T)
        return A, residuals

def k_means(X, k):
    X = np.array(X)
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)

    # Get the cluster centers (representative matrices)
    cluster_centers = kmeans.cluster_centers_

    # Assign each matrix to its nearest cluster
    labels, _ = pairwise_distances_argmin_min(X, cluster_centers)

    # Print the cluster assignments for each matrix
    for i, label in enumerate(labels):
        print(f"Matrix {i + 1} is assigned to Cluster {label + 1}")
def cos_distance_matrix(X):
    D = np.zeros((X.shape[0], X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            print(i, j, X.shape[0])
            D[i][j] = spatial.distance.cosine(X[i], X[j])
    return D


In [3]:
pairwise_evaluation = json.load(open('pairwise_evaluation_w_embeddings.json'))
transformations = []

for sample in pairwise_evaluation:
    # A1, residuals = estimate_transformation(sample['full_embedding'], sample['llm_summary_embedding'])
    # transformations.append(A1)
    A2, residuals = estimate_transformation(sample['full_embedding'], sample['writer_summary_embedding'])
    transformations.append(A2.flatten())
print("estimation done")
X = np.array(transformations)
D = cos_distance_matrix(X)
print(X.shape, D.shape)
# k_means(X, 5)

estimation done
